# Device configuration data (DCD)

This example shows various ways how to create/parse Device Configuration Data (DCD) commands sequence config for pre-initialization of devices at boot time.

You might also refer to MCUXpresso Config Tools that offers DCD tool [MCUXpresso Config Tools](https://www.nxp.com/design/software/development-software/mcuxpresso-software-and-tools-/mcuxpresso-config-tools-pins-clocks-and-peripherals:MCUXpresso-Config-Tools)

In [1]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
#
# Copyright 2019-2023 NXP
#
# SPDX-License-Identifier: BSD-3-Clause

from spsdk.image.segments import (
    CmdCheckData,
    CmdNop,
    CmdWriteData,
    EnumCheckOps,
    EnumWriteOps,
    SegDCD,
)
from spsdk.utils.misc import write_file

TEMP_DIR = "workspace/"  # change this to path to your workspace


# Example-01: Create DCD object from TXT data
def dcd_from_txt() -> SegDCD:
    """Create DCD from a text (text file)."""
    data = """
        WriteValue    4 0x30340004 0x4F400005
        WriteValue    4 0x30340004 0x4F400005
        WriteValue    4 0x30340004 0x4F400005
        WriteValue    4 0x30340004 0x4F400005
        ClearBitMask  4 0x307900C4 0x00000001
        SetBitMask    4 0x307900C4 0x00000001
        CheckAllClear 4 0x307900C4 0x00000001
        CheckAllClear 4 0x307900C4 0x00000001 5
        CheckAnyClear 4 0x307900C4 0x00000001
        CheckAnyClear 4 0x307900C4 0x00000001 5
        CheckAllSet   4 0x307900C4 0x00000001
        CheckAllSet   4 0x307900C4 0x00000001 5
        CheckAnySet   4 0x307900C4 0x00000001
        CheckAnySet   4 0x307900C4 0x00000001 5
        Nop
    """
    return SegDCD.parse_txt(data)


# Example-02: Create DCD object from BIN data
def dcd_from_bin() -> SegDCD:
    """Create DCD from binary data (from binary file)."""
    # fmt: off
    data = bytes([
        0xd2, 0x00, 0xb4, 0x41, 0xcc, 0x00, 0x24, 0x04, 0x30, 0x34, 0x00, 0x04, 0x4f, 0x40, 0x00, 0x05,
        0x30, 0x34, 0x00, 0x04, 0x4f, 0x40, 0x00, 0x05, 0x30, 0x34, 0x00, 0x04, 0x4f, 0x40, 0x00, 0x05,
        0x30, 0x34, 0x00, 0x04, 0x4f, 0x40, 0x00, 0x05, 0xcc, 0x00, 0x0c, 0x14, 0x30, 0x79, 0x00, 0xc4,
        0x00, 0x00, 0x00, 0x01, 0xcc, 0x00, 0x0c, 0x1c, 0x30, 0x79, 0x00, 0xc4, 0x00, 0x00, 0x00, 0x01,
        0xcf, 0x00, 0x0c, 0x04, 0x30, 0x79, 0x00, 0xc4, 0x00, 0x00, 0x00, 0x01, 0xcf, 0x00, 0x10, 0x04,
        0x30, 0x79, 0x00, 0xc4, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x05, 0xcf, 0x00, 0x0c, 0x14,
        0x30, 0x79, 0x00, 0xc4, 0x00, 0x00, 0x00, 0x01, 0xcf, 0x00, 0x10, 0x14, 0x30, 0x79, 0x00, 0xc4,
        0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x05, 0xcf, 0x00, 0x0c, 0x0c, 0x30, 0x79, 0x00, 0xc4,
        0x00, 0x00, 0x00, 0x01, 0xcf, 0x00, 0x10, 0x0c, 0x30, 0x79, 0x00, 0xc4, 0x00, 0x00, 0x00, 0x01,
        0x00, 0x00, 0x00, 0x05, 0xcf, 0x00, 0x0c, 0x1c, 0x30, 0x79, 0x00, 0xc4, 0x00, 0x00, 0x00, 0x01,
        0xcf, 0x00, 0x10, 0x1c, 0x30, 0x79, 0x00, 0xc4, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x05,
        0xc0, 0x00, 0x04, 0x00,
    ])
    # fmt: on
    return SegDCD.parse(data)


# Example-03: DCD object scripted in python
def dcd_in_python() -> SegDCD:
    """Create DCD in python code."""
    dcd = SegDCD(enabled=True)
    dcd.append(
        CmdWriteData(
            ops=EnumWriteOps.WRITE_VALUE,
            data=(
                (0x30340004, 0x4F400005),
                (0x30340004, 0x4F400005),
                (0x30340004, 0x4F400005),
                (0x30340004, 0x4F400005),
            ),
        )
    )
    dcd.append(CmdWriteData(ops=EnumWriteOps.CLEAR_BITMASK, data=((0x307900C4, 0x00000001),)))
    dcd.append(CmdWriteData(ops=EnumWriteOps.SET_BITMASK, data=((0x307900C4, 0x00000001),)))
    dcd.append(CmdCheckData(ops=EnumCheckOps.ALL_CLEAR, address=0x307900C4, mask=0x00000001))
    dcd.append(
        CmdCheckData(ops=EnumCheckOps.ALL_CLEAR, address=0x307900C4, mask=0x00000001, count=5)
    )
    dcd.append(CmdCheckData(ops=EnumCheckOps.ANY_CLEAR, address=0x307900C4, mask=0x00000001))
    dcd.append(
        CmdCheckData(ops=EnumCheckOps.ANY_CLEAR, address=0x307900C4, mask=0x00000001, count=5)
    )
    dcd.append(CmdCheckData(ops=EnumCheckOps.ALL_SET, address=0x307900C4, mask=0x00000001))
    dcd.append(CmdCheckData(ops=EnumCheckOps.ALL_SET, address=0x307900C4, mask=0x00000001, count=5))
    dcd.append(CmdCheckData(ops=EnumCheckOps.ANY_SET, address=0x307900C4, mask=0x00000001))
    dcd.append(CmdCheckData(ops=EnumCheckOps.ANY_SET, address=0x307900C4, mask=0x00000001, count=5))
    dcd.append(CmdNop())
    return dcd


def main() -> None:
    """Main function."""
    dcd_01 = dcd_from_txt()
    dcd_02 = dcd_from_bin()
    dcd_03 = dcd_in_python()

    # All DCD objects contain same data, therefore must be same
    print(dcd_01 == dcd_02 == dcd_03)
    print(str(dcd_01))

    # Store DCD object into TXT file
    write_file(dcd_01.export_txt(), f"{TEMP_DIR}/dcd.txt")

    # Store DCD object into BIN file
    write_file(dcd_01.export(), f"{TEMP_DIR}/dcd.bin", "wb")


if __name__ == "__main__":
    main()

True
------------------------------------------------------------
Command "Write Data"   [Tag=204, length=36]
Write Data Command (Ops: WRITE_VALUE, Bytes: 4)
- Address: 0x30340004, Value: 0x4F400005
- Address: 0x30340004, Value: 0x4F400005
- Address: 0x30340004, Value: 0x4F400005
- Address: 0x30340004, Value: 0x4F400005
------------------------------------------------------------

------------------------------------------------------------
Command "Write Data"   [Tag=204, length=12]
Write Data Command (Ops: CLEAR_BITMASK, Bytes: 4)
- Address: 0x307900C4, Value: 0x00000001
------------------------------------------------------------

------------------------------------------------------------
Command "Write Data"   [Tag=204, length=12]
Write Data Command (Ops: SET_BITMASK, Bytes: 4)
- Address: 0x307900C4, Value: 0x00000001
------------------------------------------------------------

------------------------------------------------------------
Command "Check Data"   [Tag=207, length=1